In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup

In [3]:
# # test request
# # webpage that require subscription
# # url = "https://www.sciencedirect.com/science/article/abs/pii/S0079612305490019?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S095943880900049X?via%3Dihub"
# # webpage that does not require subscription
# # url = "https://pubmed.ncbi.nlm.nih.gov/34524542/"
# # url = "https://link.springer.com/article/10.1007/s00429-021-02377-7"
# response = requests.get(url, headers = plib.headers)
# soup = BeautifulSoup(response.content,"html.parser")
# # print(soup.find_all("a"))
# print(soup)

In [4]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# # from selenium.webdriver.chrome.options import Options

# # options = Options()
# # options.add_argument('--headless')
# browser = webdriver.Firefox()

# browser.get(url)
# time.sleep(10)
# # assert 'Yahoo' in browser.title

# # ele = browse?r.find_element("a")
# # elems = browser.find_element(By.CLASS_NAME, "title-text")
# # pdf_links = browser.find_elements(By.XPATH, "//a[contains(@href, '.pdf')]")
# pdf_links = browser.find_elements(By.XPATH, "//h1")[0].get_attribute('innerHTML')
# print(pdf_links)
# # for elem in elems:
# #     print(elem)
# # print(elems)
# # elem = browser.find_element(By.NAME, 'p')  # Find the search box
# # elem.send_keys('seleniumhq' + Keys.RETURN)
# browser.quit()